<a href="https://colab.research.google.com/github/Collin-Campbell/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/LS_DS_232_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# https://www.kaggle.com/fema/federal-disasters

df = pd.read_csv('database.csv', parse_dates=['Declaration Date'])

In [ ]:
print(df.shape)
df.head()

(46185, 14)


,Declaration Number,Declaration Type,Declaration Date,State,County,Disaster Type,Disaster Title,Start Date,End Date,Close Date,Individual Assistance Program,Individuals & Households Program,Public Assistance Program,Hazard Mitigation Program
0,DR-1,Disaster,1953-05-02,GA,NaN,Tornado,Tornado,05/02/1953,05/02/1953,06/01/1954,Yes,No,Yes,Yes
1,DR-2,Disaster,1953-05-15,TX,NaN,Tornado,Tornado and Heavy Rainfall,05/15/1953,05/15/1953,01/01/1958,Yes,No,Yes,Yes
2,DR-3,Disaster,1953-05-29,LA,NaN,Flood,Flood,05/29/1953,05/29/1953,02/01/1960,Yes,No,Yes,Yes
3,DR-4,Disaster,1953-06-02,MI,NaN,Tornado,Tornado,06/02/1953,06/02/1953,02/01/1956,Yes,No,Yes,Yes
4,DR-5,Disaster,1953-06-06,MT,NaN,Flood,Floods,06/06/1953,06/06/1953,12/01/1955,Yes,No,Yes,Yes


In [ ]:
df_emergencies = df['Declaration Date'].drop_duplicates()

assert len(df_emergencies) == df_emergencies.nunique()

In [ ]:
df_emergencies.to_csv('emergencies.csv')

In [ ]:
# https://www.kaggle.com/lsind18/daily-london-metal-fix-prices

df1 = pd.read_csv('londonfixes-current-clean_1990_.csv', 
                  parse_dates=['Date'], 
                  index_col='Date')

df2 = pd.read_csv('emergencies.csv', 
                  parse_dates=['Declaration Date'], 
                  index_col='Declaration Date')

df2['Emergency_Declared'] = 'Yes'

df2 = df2.drop(columns='Unnamed: 0')

In [ ]:
print(df1.shape)
df1.head()

(7617, 7)


,Gold AM Fix,Gold PM Fix,Silver Fix,Platinum AM Fix,Platinum PM Fix,Palladium AM Fix,Palladium PM Fix
Date,,,,,,,
1990-04-02,368.00,368.40,4.9375,471.00,470.50,128.00,127.65
1990-04-03,374.80,375.00,5.0010,475.80,477.25,128.35,128.50
1990-04-04,374.25,374.25,5.0100,475.70,476.75,128.35,128.00
1990-04-05,378.10,378.40,5.1475,481.75,481.85,128.40,127.75
1990-04-06,380.35,377.70,5.1790,481.00,480.25,128.75,128.50


In [ ]:
print(df2.shape)
df2.head()

(2507, 1)


,Emergency_Declared
Declaration Date,
1953-05-02,Yes
1953-05-15,Yes
1953-05-29,Yes
1953-06-02,Yes
1953-06-06,Yes


In [ ]:
df_merged = pd.merge(df1, df2, left_index=True, right_index=True, how='outer')

df_merged['Emergency_Declared'].fillna(value='No', inplace=True)

df_merged.dropna(inplace=True)

df_merged = df_merged[df_merged.index < '2017-02-15']

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6728 entries, 1990-04-02 to 2017-02-14
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Gold AM Fix         6728 non-null   float64
 1   Gold PM Fix         6728 non-null   float64
 2   Silver Fix          6728 non-null   float64
 3   Platinum AM Fix     6728 non-null   float64
 4   Platinum PM Fix     6728 non-null   float64
 5   Palladium AM Fix    6728 non-null   float64
 6   Palladium PM Fix    6728 non-null   float64
 7   Emergency_Declared  6728 non-null   object 
dtypes: float64(7), object(1)
memory usage: 473.1+ KB


In [ ]:
print(df_merged.shape)
df_merged.head()

(6728, 8)


,Gold AM Fix,Gold PM Fix,Silver Fix,Platinum AM Fix,Platinum PM Fix,Palladium AM Fix,Palladium PM Fix,Emergency_Declared
1990-04-02,368.00,368.40,4.9375,471.00,470.50,128.00,127.65,No
1990-04-03,374.80,375.00,5.0010,475.80,477.25,128.35,128.50,Yes
1990-04-04,374.25,374.25,5.0100,475.70,476.75,128.35,128.00,No
1990-04-05,378.10,378.40,5.1475,481.75,481.85,128.40,127.75,No
1990-04-06,380.35,377.70,5.1790,481.00,480.25,128.75,128.50,No


In [ ]:
6728 * 0.80

5382.400000000001

In [ ]:
df_merged.index[5382]

Timestamp('2011-10-03 00:00:00')

In [ ]:
# Splitting into target and feature matrix

target = 'Emergency_Declared'

y = df_merged[target]
X = df_merged.drop(columns=target)

In [ ]:
# Splitting into training and test sets:

cutoff = '2011-10-03'
mask = X.index < cutoff

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

In [ ]:
assert len(X_train) + len(X_test) == len(X)

In [ ]:
# Establishing baseline (classification problem, so majority class)

baseline_acc = y_train.value_counts(normalize=True).max()

print('Baseline Accuracy:', baseline_acc)

Baseline Accuracy: 0.771832032701598


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

# Building model pipeline

forest_model = make_pipeline(
      SimpleImputer(),
      RandomForestClassifier(random_state=42,
                            n_jobs=-1,
      ))

In [ ]:
forest_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=-1,
                                        oob_score=Fal

In [ ]:
# Checking metrics

print('FOREST: Training Accuracy:', forest_model.score(X_train, y_train))
print('FOREST: Testing Accuracy:', forest_model.score(X_test, y_test))

FOREST: Training Accuracy: 0.9998141954663694
FOREST: Testing Accuracy: 0.789261744966443
